# PyDipAPI - Datenvisualisierung Tutorial

Dieses Notebook zeigt, wie Sie Bundestag-Daten visualisieren.

## Inhalt:
1. **Matplotlib** - Statische Diagramme
2. **Datenanalyse** - Parteien und Statistiken
3. **Export-Funktionen** - Diagramme speichern
4. **Dashboard-Konzepte** - Übersichtsdarstellungen

In [2]:
import sys
import os
from collections import Counter
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage

try:
    import matplotlib.pyplot as plt
    print("✅ Matplotlib verfügbar")
except ImportError:
    print("📦 Installiere matplotlib...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib"])
    import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

API_KEY = "OSOegLs.PR2lwJ1dwCeje9vTj7FPOt3hvpYKtwKkhw"
dip = DipAnfrage(api_key=API_KEY)

print("✅ Datenvisualisierung Tutorial bereit!")

✅ Matplotlib verfügbar
✅ Datenvisualisierung Tutorial bereit!


## Daten laden

In [3]:
print("📊 Lade Daten...")

try:
    persons_data = dip.get_person(anzahl=30)
    print(f"✅ {len(persons_data)} Personen geladen")
    
    documents_data = dip.get_drucksache(anzahl=20)
    print(f"✅ {len(documents_data)} Dokumente geladen")
    
except Exception as e:
    print(f"❌ Fehler: {e}")
    persons_data = []
    documents_data = []

📊 Lade Daten...
✅ 30 Personen geladen
✅ 20 Dokumente geladen


## Parteienverteilung visualisieren

In [4]:
if persons_data:
    # Parteien extrahieren
    parties = []
    for person in persons_data:
        party = (person.get('fraktion') or 
                person.get('partei') or 
                'Unbekannt')
        parties.append(party)
    
    party_counts = Counter(parties)
    
    print("🏛️ Parteienverteilung:")
    for party, count in party_counts.most_common():
        print(f"  {party}: {count} Personen")
    
    # Balkendiagramm
    plt.figure(figsize=(12, 6))
    parties_list = list(party_counts.keys())
    counts_list = list(party_counts.values())
    
    bars = plt.bar(parties_list, counts_list, color='steelblue')
    plt.title('Parteienverteilung', fontsize=16, fontweight='bold')
    plt.xlabel('Partei/Fraktion')
    plt.ylabel('Anzahl Personen')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # Werte auf Balken
    for bar, count in zip(bars, counts_list):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                str(count), ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('parteien_verteilung.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Diagramm gespeichert")
else:
    print("❌ Keine Personendaten verfügbar")

TypeError: unhashable type: 'list'

## Zusammenfassung

✅ **Datenvisualisierung Tutorial abgeschlossen!**

### Funktionen:
- **Matplotlib** - Statische, hochwertige Diagramme
- **Datenanalyse** - Parteienverteilung und Trends
- **Export** - Diagramme als PNG speichern

**🎉 Alle PyDipAPI Tutorials erfolgreich abgeschlossen!**